# Evaluations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import geopandas as gpd

def calculate_precision_recall_f1(confussion_matix):
    """
    Calculate precisoin, recall, and F1 based on the confussion matrix (TP, FP, FN)

    Args:
        confussion_matrix (tuple): confussion matrix values consisting TP, FP, FN.

    Returns:
        tuple: Precision and Recall scores.
    """
    # Confussion matrix values slicing
    true_positives, false_positives, false_negatives = confussion_matix

    # Calculate precision, recall, and F1
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

In [ ]:
def prediction_evaluation(prediction, ground_truth, prcat_path, gtcat_path, iou_threshold=0.5):
    """
    Compare prediction results and ground truth polygons and returns ocnfussion matrix values.
    It aslo creates a new column for the category of the polygon for the visualization purpose.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        prcat_path (String): categorized prediction shapefile output filename and path
        gtcat_path (String): categorized grount truth shapefile output filename and path
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).

    Returns:
        tuple: confussion matrix values (TP, FP, FN).
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    # Convert polygons to lists for pairwise comparison
    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    # Create a copy of prediction's and ground_truth's geodataframe
    # prediction_mod = prediction.copy()
    # prediction_mod = prediction_mod.assign(cat=0) # category column
    # prediction_mod = prediction_mod.assign(iou=0.0) # iou column
    # prediction_mod = prediction_mod.assign(gt_match=-1) # match column
    # groundtr_mod = ground_truth.copy()
    # groundtr_mod = groundtr_mod.assign(cat=0) # category column
    # groundtr_mod = groundtr_mod.assign(iou=0.0) # iou column
    # groundtr_mod = groundtr_mod.assign(gt_match=-1) # match column
    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["iou"] = 0.0 # iou column
    prediction_mod["gt_match"]= -1 # match column
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["iou"] = 0.0 # iou column
    groundtr_mod["gt_match"]= -1 # match column


    # Track matches
    matched_gt = set()  # Track matched ground truth indices
    duplicated_polys = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    iou_list = []

    # Check each predicted polygon
    for p_idx, pred_poly in enumerate(predicted_polys):
        matched = False
        duplicated = False
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0
            prediction_mod.loc[p_idx, 'iou'] = iou # For IoU

            # Check if IoU exceeds threshold
            if iou >= iou_threshold:
                matched = True
                matched_gt.add(idx)  # Mark this ground truth as matched
                groundtr_mod.loc[idx, 'cat'] = 2 # Category 2 for Matched
                if groundtr_mod.loc[idx, 'gt_match'] != -1:
                    duplicated = True
                if groundtr_mod.loc[idx, 'gt_match'] == -1 or groundtr_mod.loc[idx, 'iou'] < iou: #Store the matching prediction
                    groundtr_mod.loc[idx, 'iou'] = iou
                    groundtr_mod.loc[idx, 'gt_match'] = p_idx #
                    # duplicated_polys += 1
                prediction_mod.loc[p_idx, 'gt_match'] = idx # Store the matching ground truth polygon
                break


        if matched:
            true_positives += 1
            prediction_mod.loc[p_idx, 'cat'] = 2 # Category 2 for TP
        else:
            false_positives += 1
            # prediction_mod.loc[p_idx, 'cat'] = 1 # Category 1 for FP

        if duplicated:
            duplicated_polys += 1

    # Re-Count the True positives and False positives considering the duplication
    true_positives = true_positives - duplicated_polys
    false_positives = false_positives

    # Count false negatives (unmatched ground truth polygons)
    false_negatives = len(ground_truth_polys) - len(matched_gt)

    print("True Positives", true_positives)
    print("False Positives", false_positives)
    print("False Negatives", false_negatives)

    prediction_mod.to_file(prcat_path)
    groundtr_mod.to_file(gtcat_path)

    return true_positives, false_positives, false_negatives

In [ ]:
from shapely.ops import unary_union

def calculate_iou(prediction, ground_truth):
    """
    Calculate IoU between prediction and ground truth polygons.

    Args:
        prediction (GeoDataFrame): Predicted polygons.
        ground_truth (GeoDataFrame): Ground truth polygons.

    Returns:
        float: IoU score between prediction and ground truth.
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    # Merge all polygons in prediction and ground truth
    merged_prediction = unary_union(prediction.geometry)
    merged_ground_truth = unary_union(ground_truth.geometry)

    # Calculate intersection and union areas
    intersection_area = merged_prediction.intersection(merged_ground_truth).area
    union_area = merged_prediction.union(merged_ground_truth).area

    # IoU calculation
    iou = intersection_area / union_area if union_area > 0 else 0

    return iou

### Check IoU

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "jw_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L4.gpkg"
gt_in = "ls_plot_4_gt.gpkg"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
ground_truth = gpd.read_file(gt_file)
prediction = gpd.read_file(pr_file)

iou = calculate_iou(prediction, ground_truth)
print(f"IoU between prediction and ground truth: {iou}")

IoU between prediction and ground truth: 0.3270928968399146


### ================================

### Check evaluation L0

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "jw_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L4.gpkg"
gt_in = "ls_plot_4_gt.gpkg"

out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
# Load ground truth shapefile
pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

In [ ]:
cf_matrix = prediction_evaluation(pr_crown, gt_crown, out_pr, out_gt)
precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)
iou = calculate_iou(pr_crown, gt_crown)
print(f"IoU: {iou:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

True Positives 15
False Positives 51
False Negatives 7


/tmp/ipython-input-128-773889458.py:99: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  prediction_mod.to_file(prcat_path)
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Confidence_score' to 'Confidence'
  ogr_write(


IoU: 0.3271
Precision: 0.2273
Recall: 0.6818
F1 Score: 0.3409


### Check evaluation L1

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "jw_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L1.gpkg"
gt_in = "ls_plot_1_gt.gpkg"

out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

In [ ]:
cf_matrix = prediction_evaluation(pr_crown, gt_crown, out_pr, out_gt)
precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

/tmp/ipython-input-54-1397326828.py:96: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  prediction_mod.to_file(prcat_path)
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Confidence_score' to 'Confidence'
  ogr_write(


In [ ]:
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Precision: 0.2281
Recall: 0.5417
F1 Score: 0.3210


In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "jw_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L2.gpkg"
gt_in = "ls_plot_2_gt.gpkg"

out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

cf_matrix = prediction_evaluation(pr_crown, gt_crown, out_pr, out_gt)
precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Precision: 0.5309
Recall: 0.5658
F1 Score: 0.5478


/tmp/ipython-input-54-1397326828.py:96: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  prediction_mod.to_file(prcat_path)
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Confidence_score' to 'Confidence'
  ogr_write(


In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "jw_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L4.gpkg"
gt_in = "ls_plot_4_gt.gpkg"

out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

cf_matrix = prediction_evaluation(pr_crown, gt_crown, out_pr, out_gt)
precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

Precision: 0.2273
Recall: 0.6818
F1 Score: 0.3409


/tmp/ipython-input-54-1397326828.py:96: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  prediction_mod.to_file(prcat_path)
/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Confidence_score' to 'Confidence'
  ogr_write(


### Otomate LS

In [ ]:
num_code = ["0", "2", "4"]

In [ ]:
ls_name = [
    "ls_resnet_L",
    "ls_resnext_L",
    "ls_resnet_pr_L",
    "ls_resnext_pr_L"]

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "ls_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    print("================================")


ls_resnet_L
Results for 0
True Positives 29
False Positives 24
False Negatives 17
IoU: 0.6651
Precision: 0.5472
Recall: 0.6304
F1 Score: 0.5859
Results for 2
True Positives 39
False Positives 21
False Negatives 37
IoU: 0.5001
Precision: 0.6500
Recall: 0.5132
F1 Score: 0.5735
Results for 4
True Positives 11
False Positives 21
False Negatives 11
IoU: 0.3267
Precision: 0.3438
Recall: 0.5000
F1 Score: 0.4074

ls_resnext_L
Results for 0
True Positives 30
False Positives 16
False Negatives 16
IoU: 0.6605
Precision: 0.6522
Recall: 0.6522
F1 Score: 0.6522
Results for 2
True Positives 41
False Positives 20
False Negatives 35
IoU: 0.5516
Precision: 0.6721
Recall: 0.5395
F1 Score: 0.5985
Results for 4
True Positives 15
False Positives 27
False Negatives 7
IoU: 0.2817
Precision: 0.3571
Recall: 0.6818
F1 Score: 0.4688

ls_resnet_pr_L
Results for 0
True Positives 24
False Positives 5
False Negatives 22
IoU: 0.4718
Precision: 0.8276
Recall: 0.5217
F1 Score: 0.6400
Results for 2
True Positives 33
Fal

### Otomate JW

In [ ]:
num_code = ["2"]

In [ ]:
ls_name = [
    "jw_resnet_P",
    "jw_resnext_P",
    "jw_resnet_pr_P",
    "jw_resnext_pr_P"]

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "jw_res/"
gt_path = site_path + "ground_truth/"

In [ ]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "jw_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    print("================================")


jw_resnet_P
Results for 2
True Positives 35
False Positives 12
False Negatives 31
IoU: 0.4733
Precision: 0.7447
Recall: 0.5303
F1 Score: 0.6195

jw_resnext_P
Results for 2
True Positives 44
False Positives 20
False Negatives 22
IoU: 0.5971
Precision: 0.6875
Recall: 0.6667
F1 Score: 0.6769

jw_resnet_pr_P
Results for 2
True Positives 32
False Positives 4
False Negatives 34
IoU: 0.4101
Precision: 0.8889
Recall: 0.4848
F1 Score: 0.6275

jw_resnext_pr_P
Results for 2
True Positives 30
False Positives 4
False Negatives 36
IoU: 0.4155
Precision: 0.8824
Recall: 0.4545
F1 Score: 0.6000


### Modified

In [ ]:
def prediction_evaluation2(prediction, ground_truth, prcat_path, gtcat_path, iou_threshold=0.5):
    """
    Compare prediction results and ground truth polygons and returns ocnfussion matrix values.
    It aslo creates a new column for the category of the polygon for the visualization purpose.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        prcat_path (String): categorized prediction shapefile output filename and path
        gtcat_path (String): categorized grount truth shapefile output filename and path
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).

    Returns:
        tuple: confussion matrix values (TP, FP, FN).
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    # Convert polygons to lists for pairwise comparison
    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["iou"] = 0.0 # iou column
    prediction_mod["gt_match"]= -1 # match column
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["iou"] = 0.0 # iou column
    groundtr_mod["gt_match"]= -1 # match column


    # Track matches
    matched_gt = set()  # Track matched ground truth indices
    duplicated_polys = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    iou_list = []

    # Check each predicted polygon
    for p_idx, pred_poly in enumerate(predicted_polys):
        matched = False
        duplicated = False
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0
            prediction_mod.loc[p_idx, 'iou'] = iou # For IoU

            # Check if IoU exceeds threshold
            if iou >= iou_threshold:
                matched = True
                iou_list.append(iou)
                matched_gt.add(idx)  # Mark this ground truth as matched
                groundtr_mod.loc[idx, 'cat'] = 2 # Category 2 for Matched
                if groundtr_mod.loc[idx, 'gt_match'] != -1:
                    duplicated = True
                if groundtr_mod.loc[idx, 'gt_match'] == -1 or groundtr_mod.loc[idx, 'iou'] < iou: #Store the matching prediction
                    groundtr_mod.loc[idx, 'iou'] = iou
                    groundtr_mod.loc[idx, 'gt_match'] = p_idx #
                    # duplicated_polys += 1
                prediction_mod.loc[p_idx, 'gt_match'] = idx # Store the matching ground truth polygon
                break


        if matched:
            true_positives += 1
            prediction_mod.loc[p_idx, 'cat'] = 2 # Category 2 for TP
        else:
            false_positives += 1
            # prediction_mod.loc[p_idx, 'cat'] = 1 # Category 1 for FP

        if duplicated:
            duplicated_polys += 1

    # Re-Count the True positives and False positives considering the duplication
    true_positives = true_positives - duplicated_polys
    false_positives = false_positives

    # Count false negatives (unmatched ground truth polygons)
    false_negatives = len(ground_truth_polys) - len(matched_gt)

    print("True Positives", true_positives)
    print("False Positives", false_positives)
    print("False Negatives", false_negatives)
    # print(iou_list)
    avg_iou = sum(iou_list)/len(iou_list)
    print(f"Average IoU: {avg_iou:.4f}")

    prediction_mod.to_file(prcat_path)
    groundtr_mod.to_file(gtcat_path)

    return true_positives, false_positives, false_negatives

### ResNet L0

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L0.gpkg"
gt_in = "ls_plot_0_gt.gpkg"

out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
# Load ground truth shapefile
pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

In [ ]:
cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)

True Positives 29
False Positives 24
False Negatives 17
[0.6045179734623213, 0.8608954795957594, 0.8570672058370433, 0.5793564096054145, 0.8330687103278542, 0.8147718496751322, 0.7079184710333389, 0.9269574529122351, 0.8513104169138087, 0.8948594051834834, 0.7064728509611568, 0.9322411503804541, 0.8935881186242111, 0.9154545062557037, 0.8578296120749977, 0.7247905820359352, 0.70639459937713, 0.8950659759095827, 0.8814556731487204, 0.8405967919678051, 0.8948510001304697, 0.8427553253697997, 0.8348359064974833, 0.7956577079009421, 0.8651604369876347, 0.8869311808845515, 0.6749721248769561, 0.9155716191130618, 0.7347778326702868]
Average IoU: 0.8182802196452854


### ResNeXt L0

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnext_L0.gpkg"
gt_in = "ls_plot_0_gt.gpkg"

out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
# Load ground truth shapefile
pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

In [ ]:
cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)

True Positives 30
False Positives 16
False Negatives 16
[0.861914049031583, 0.882541594900042, 0.7811073471947628, 0.8185320589464278, 0.7270189308866201, 0.7386205136195401, 0.7034353891965947, 0.5462510164960686, 0.7883369548409427, 0.9281619918268516, 0.7209064452380183, 0.6170113125394614, 0.9157578478992844, 0.8922762018142526, 0.9248976608998511, 0.7438957576446403, 0.843786179790097, 0.5065988515508416, 0.8482406477729691, 0.9188985646606899, 0.8841076181053131, 0.7650968692162627, 0.78091949709176, 0.8427862366591798, 0.8364602534976888, 0.7863830343918159, 0.8897196723904962, 0.6545801697181076, 0.9078215081695146, 0.7167931575570268]
Average IoU: 0.7924285777848901


### ResNet + PointRend L0

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_pr_L0.gpkg"
gt_in = "ls_plot_0_gt.gpkg"

out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
# Load ground truth shapefile
pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

In [ ]:
cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)

True Positives 24
False Positives 5
False Negatives 22
[0.8762289392891961, 0.6507614995421075, 0.868303681513267, 0.7483116625875785, 0.5771481317515538, 0.7879912074546445, 0.607895380164598, 0.9348178849630542, 0.9307116231933135, 0.9448538813940451, 0.9137691738931449, 0.5097791079774066, 0.8099714747535584, 0.853719983262897, 0.9065166516269927, 0.8359173647885585, 0.8743960926387295, 0.7725543547433494, 0.8128137640003307, 0.874789598783093, 0.8119586884262016, 0.906950396152686, 0.667669517618395, 0.7172056737659367]
Average IoU: 0.7997931555951933


### ResNeXt + PointRend L0

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnext_pr_L0.gpkg"
gt_in = "ls_plot_0_gt.gpkg"

out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
# Load ground truth shapefile
pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

In [ ]:
cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)

True Positives 23
False Positives 3
False Negatives 23
[0.8486526326459107, 0.929034175873213, 0.8378604665758366, 0.6546855153841681, 0.9295705426882317, 0.8713156373656687, 0.8586500290061078, 0.9000108885396761, 0.9338201262845228, 0.67298507621302, 0.7198087313942954, 0.5013930572751325, 0.8772999692012768, 0.900135647798076, 0.8760538677106392, 0.879741834350766, 0.8695956328423347, 0.7220118503563506, 0.7935419104297073, 0.9208729440754044, 0.6976914562098202, 0.9009575268890314, 0.8275077283160872, 0.6605618093617421]
Average IoU: 0.8159899606994592


### Otomate LS

In [ ]:
num_code = ["0", "2", "4"]

In [ ]:
ls_name = [
    "ls_resnet_L",
    "ls_resnext_L",
    "ls_resnet_pr_L",
    "ls_resnext_pr_L"]

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "ls_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    print("================================")


ls_resnet_L
Results for 0
True Positives 29
False Positives 24
False Negatives 17
Average IoU: 0.8183
IoU: 0.6651
Precision: 0.5472
Recall: 0.6304
F1 Score: 0.5859
Results for 2
True Positives 39
False Positives 21
False Negatives 37
Average IoU: 0.8143
IoU: 0.5001
Precision: 0.6500
Recall: 0.5132
F1 Score: 0.5735
Results for 4
True Positives 11
False Positives 21
False Negatives 11
Average IoU: 0.7951
IoU: 0.3267
Precision: 0.3438
Recall: 0.5000
F1 Score: 0.4074

ls_resnext_L
Results for 0
True Positives 30
False Positives 16
False Negatives 16
Average IoU: 0.7924
IoU: 0.6605
Precision: 0.6522
Recall: 0.6522
F1 Score: 0.6522
Results for 2
True Positives 41
False Positives 20
False Negatives 35
Average IoU: 0.7727
IoU: 0.5516
Precision: 0.6721
Recall: 0.5395
F1 Score: 0.5985
Results for 4
True Positives 15
False Positives 27
False Negatives 7
Average IoU: 0.7442
IoU: 0.2817
Precision: 0.3571
Recall: 0.6818
F1 Score: 0.4688

ls_resnet_pr_L
Results for 0
True Positives 24
False Positive

### Otomate LS - Exp ResNet Pr

In [ ]:
num_code = ["0", "2", "4"]

In [ ]:
ls_name = [
    "ls_resnet_pr7_L",
    "ls_resnet_pr6_L"]

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "ls_exp/"
gt_path = site_path + "ground_truth/"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "ls_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    print("================================")


ls_resnet_pr7_L
Results for 0
True Positives 20
False Positives 0
False Negatives 26
Average IoU: 0.7956
IoU: 0.5954
Precision: 1.0000
Recall: 0.4348
F1 Score: 0.6061
Results for 2
True Positives 21
False Positives 3
False Negatives 55
Average IoU: 0.8467
IoU: 0.4354
Precision: 0.8750
Recall: 0.2763
F1 Score: 0.4200
Results for 4
True Positives 9
False Positives 7
False Negatives 13
Average IoU: 0.7766
IoU: 0.3918
Precision: 0.5625
Recall: 0.4091
F1 Score: 0.4737

ls_resnet_pr6_L
Results for 0
True Positives 16
False Positives 2
False Negatives 30
Average IoU: 0.7960
IoU: 0.5296
Precision: 0.8889
Recall: 0.3478
F1 Score: 0.5000
Results for 2
True Positives 24
False Positives 3
False Negatives 52
Average IoU: 0.8141
IoU: 0.4422
Precision: 0.8889
Recall: 0.3158
F1 Score: 0.4660
Results for 4
True Positives 8
False Positives 6
False Negatives 14
Average IoU: 0.7858
IoU: 0.4019
Precision: 0.5714
Recall: 0.3636
F1 Score: 0.4444


### Otomate LS - Exp ResNeXt Pr

In [ ]:
num_code = ["0", "2", "4"]

In [ ]:
ls_name = [
    "ls_resnext_pr17_L",
    "ls_resnext_pr15_L",
    "ls_resnext_pr12_L"]

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "ls_exp/"
gt_path = site_path + "ground_truth/"

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "ls_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    print("================================")


ls_resnext_pr17_L
Results for 0
True Positives 19
False Positives 1
False Negatives 27
Average IoU: 0.8286
IoU: 0.5932
Precision: 0.9500
Recall: 0.4130
F1 Score: 0.5758
Results for 2
True Positives 20
False Positives 4
False Negatives 56
Average IoU: 0.8316
IoU: 0.3942
Precision: 0.8333
Recall: 0.2632
F1 Score: 0.4000
Results for 4
True Positives 8
False Positives 2
False Negatives 14
Average IoU: 0.8326
IoU: 0.3280
Precision: 0.8000
Recall: 0.3636
F1 Score: 0.5000

ls_resnext_pr15_L
Results for 0
True Positives 21
False Positives 3
False Negatives 25
Average IoU: 0.8273
IoU: 0.4775
Precision: 0.8750
Recall: 0.4565
F1 Score: 0.6000
Results for 2
True Positives 27
False Positives 8
False Negatives 49
Average IoU: 0.7993
IoU: 0.5057
Precision: 0.7714
Recall: 0.3553
F1 Score: 0.4865
Results for 4
True Positives 11
False Positives 6
False Negatives 11
Average IoU: 0.7934
IoU: 0.3487
Precision: 0.6471
Recall: 0.5000
F1 Score: 0.5641

ls_resnext_pr12_L
Results for 0
True Positives 27
False 

### Otomate JW

In [ ]:
num_code = ["2"]

In [ ]:
ls_name = [
    "jw_resnet_P",
    "jw_resnext_P",
    "jw_resnet_pr_P",
    "jw_resnext_pr_P"]

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "jw_res/"
gt_path = site_path + "ground_truth/"

In [ ]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "jw_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    print("================================")


jw_resnet_P
Results for 2
True Positives 35
False Positives 12
False Negatives 31
Average IoU: 0.8045
IoU: 0.4733
Precision: 0.7447
Recall: 0.5303
F1 Score: 0.6195

jw_resnext_P
Results for 2
True Positives 44
False Positives 20
False Negatives 22
Average IoU: 0.8005
IoU: 0.5971
Precision: 0.6875
Recall: 0.6667
F1 Score: 0.6769

jw_resnet_pr_P
Results for 2
True Positives 32
False Positives 4
False Negatives 34
Average IoU: 0.8293
IoU: 0.4101
Precision: 0.8889
Recall: 0.4848
F1 Score: 0.6275

jw_resnext_pr_P
Results for 2
True Positives 30
False Positives 4
False Negatives 36
Average IoU: 0.8280
IoU: 0.4155
Precision: 0.8824
Recall: 0.4545
F1 Score: 0.6000


### Otomate JW - Exp ResNeXt Pr

In [ ]:
num_code = ["2"]

In [ ]:
ls_name = [
    "jw_resnext_pr17_P",
    "jw_resnext_pr15_P",
    "jw_resnext_pr12_P"]

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "jw_exp/"
gt_path = site_path + "ground_truth/"

In [ ]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "jw_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    print("================================")


jw_resnext_pr17_P
Results for 2
True Positives 30
False Positives 2
False Negatives 36
Average IoU: 0.8478
IoU: 0.4641
Precision: 0.9375
Recall: 0.4545
F1 Score: 0.6122

jw_resnext_pr15_P
Results for 2
True Positives 30
False Positives 4
False Negatives 36
Average IoU: 0.8347
IoU: 0.4436
Precision: 0.8824
Recall: 0.4545
F1 Score: 0.6000

jw_resnext_pr12_P
Results for 2
True Positives 36
False Positives 8
False Negatives 30
Average IoU: 0.8091
IoU: 0.4822
Precision: 0.8182
Recall: 0.5455
F1 Score: 0.6545


### Otomate JW - Exp ResNet Pr

In [ ]:
num_code = ["2"]

In [ ]:
ls_name = [
    "jw_resnet_pr7_P",
    "jw_resnet_pr6_P"]

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "jw_exp/"
gt_path = site_path + "ground_truth/"

In [ ]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "jw_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

    print("================================")


jw_resnet_pr7_P
Results for 2
True Positives 28
False Positives 2
False Negatives 38
Average IoU: 0.8218
IoU: 0.4805
Precision: 0.9333
Recall: 0.4242
F1 Score: 0.5833

jw_resnet_pr6_P
Results for 2
True Positives 25
False Positives 4
False Negatives 41
Average IoU: 0.8343
IoU: 0.3969
Precision: 0.8621
Recall: 0.3788
F1 Score: 0.5263


### Over and Under

In [ ]:
def prediction_evaluation3(prediction, ground_truth, prcat_path, gtcat_path, iou_threshold=0.5, error_iou_th=0.3):
    """
    Compare prediction results and ground truth polygons and returns ocnfussion matrix values.
    It aslo creates a new column for the category of the polygon for the visualization purpose.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        prcat_path (String): categorized prediction shapefile output filename and path
        gtcat_path (String): categorized grount truth shapefile output filename and path
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).

    Returns:
        tuple: confussion matrix values (TP, FP, FN).
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    # Convert polygons to lists for pairwise comparison
    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["iou"] = 0.0 # iou column
    prediction_mod["gt_match"]= -1 # match column
    prediction_mod["under"]= 0 # match column
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["iou"] = 0.0 # iou column
    groundtr_mod["gt_match"]= -1 # match column
    groundtr_mod["over"]= 0 # match column


    # Track matches
    matched_gt = set()  # Track matched ground truth indices
    duplicated_polys = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    iou_list = []

    # Check each predicted polygon
    for p_idx, pred_poly in enumerate(predicted_polys):
        matched = False
        duplicated = False
        match_count = 0
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0
            prediction_mod.loc[p_idx, 'iou'] = iou # For IoU



            # Check if IoU exceeds threshold
            if iou >= iou_threshold:
                matched = True
                iou_list.append(iou)
                matched_gt.add(idx)  # Mark this ground truth as matched
                groundtr_mod.loc[idx, 'cat'] = 2 # Category 2 for Matched
                if groundtr_mod.loc[idx, 'gt_match'] != -1:
                    duplicated = True
                if groundtr_mod.loc[idx, 'gt_match'] == -1 or groundtr_mod.loc[idx, 'iou'] < iou: #Store the matching prediction
                    groundtr_mod.loc[idx, 'iou'] = iou
                    groundtr_mod.loc[idx, 'gt_match'] = p_idx #
                    # duplicated_polys += 1
                prediction_mod.loc[p_idx, 'gt_match'] = idx # Store the matching ground truth polygon
                break


        if matched:
            true_positives += 1
            prediction_mod.loc[p_idx, 'cat'] = 2 # Category 2 for TP
        else:
            false_positives += 1
            # prediction_mod.loc[p_idx, 'cat'] = 1 # Category 1 for FP

        if duplicated:
            duplicated_polys += 1

        prediction_mod.loc[p_idx, 'under'] = match_count

    # Re-Count the True positives and False positives considering the duplication
    true_positives = true_positives - duplicated_polys
    false_positives = false_positives

    # Count false negatives (unmatched ground truth polygons)
    false_negatives = len(ground_truth_polys) - len(matched_gt)

    print("True Positives", true_positives)
    print("False Positives", false_positives)
    print("False Negatives", false_negatives)
    # print(iou_list)
    avg_iou = sum(iou_list)/len(iou_list)
    print(f"Average IoU: {avg_iou:.4f}")

    prediction_mod.to_file(prcat_path)
    groundtr_mod.to_file(gtcat_path)

    return true_positives, false_positives, false_negatives

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L0.gpkg"
gt_in = "ls_plot_0_gt.gpkg"

# out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
# out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"
out_pr = site_path + "out_pr/" + "under_pr" + "_out.shp"
out_gt = site_path + "out_gt/" + "under_gt" + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
# Load ground truth shapefile
pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

In [ ]:
cf_matrix = prediction_evaluation3(pr_crown, gt_crown, out_pr, out_gt)

True Positives 29
False Positives 24
False Negatives 17
Average IoU: 0.8183


In [ ]:
out_file = gpd.read_file(out_pr)

In [ ]:
out_file.head(40)

,Confidence,cat,iou,gt_match,under,geometry
0,0.952046,2,0.604518,24,1,"POLYGON ((221088.098 81910.904, 221088.279 819..."
1,0.949280,2,0.860895,19,1,"POLYGON ((221103.567 81894.802, 221104.879 818..."
2,0.621935,2,0.857067,3,1,"POLYGON ((221123.513 81869.835, 221124.825 818..."
3,0.402522,0,0.000000,-1,0,"POLYGON ((221132.967 81854.05, 221133.69 81854..."
4,0.316043,0,0.000000,-1,0,"POLYGON ((221134.685 81850.296, 221134.731 818..."
5,0.265540,2,0.579356,20,1,"POLYGON ((221084.073 81878.429, 221084.977 818..."
6,0.235782,2,0.833069,34,1,"POLYGON ((221109.944 81866.081, 221110.759 818..."
7,0.234956,2,0.814772,45,1,"POLYGON ((221074.936 81873.815, 221075.75 8187..."
8,0.225033,0,0.000000,-1,1,"POLYGON ((221091.31 81909.412, 221091.355 8190..."
9,0.201611,2,0.707918,26,1,"POLYGON ((221078.736 81885.982, 221079.504 818..."


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def prediction_evaluation4(prediction, ground_truth, prcat_path, gtcat_path, iou_threshold=0.5, ov_iou_th=0.1):
    """
    Compare prediction results and ground truth polygons and returns confusion matrix values.
    It also creates a new column for the category of the polygon for visualization purpose.
    Adds oversegmentation detection.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        prcat_path (String): categorized prediction shapefile output filename and path
        gtcat_path (String): categorized ground truth shapefile output filename and path
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).

    Returns:
        tuple: confusion matrix values (TP, FP, FN), and oversegmentation count.
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    # Convert polygons to lists for pairwise comparison
    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["iou"] = 0.0 # iou column
    prediction_mod["gt_match"]= -1 # match column
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["iou"] = 0.0 # iou column
    groundtr_mod["gt_match"]= -1 # match column
    groundtr_mod["num_match"] = 0 # count of matching predictions

    # Track matches
    matched_gt = set()  # Track matched ground truth indices
    duplicated_polys = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    oversegmented = 0  # Count of oversegmented ground truth polygons

    iou_list = []

    # First pass: find all matches between predictions and ground truth
    for p_idx, pred_poly in enumerate(predicted_polys):
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0


            if iou >= ov_iou_th:
                groundtr_mod.loc[idx, 'num_match'] += 1
            # Check if IoU exceeds threshold
            if iou >= iou_threshold:
                prediction_mod.loc[p_idx, 'iou'] = max(prediction_mod.loc[p_idx, 'iou'], iou)
                prediction_mod.loc[p_idx, 'gt_match'] = idx
                # groundtr_mod.loc[idx, 'num_match'] += 1

    # Second pass: categorize predictions and count oversegmentation
    for p_idx, pred_poly in enumerate(predicted_polys):
        matched_gt_idx = prediction_mod.loc[p_idx, 'gt_match']
        if matched_gt_idx != -1:  # If this prediction matches a ground truth
            iou = prediction_mod.loc[p_idx, 'iou']
            iou_list.append(iou)

            # Check if this is the best match for the ground truth
            if groundtr_mod.loc[matched_gt_idx, 'iou'] < iou:
                # If there was a previous best match, it might now be an oversegmentation
                if groundtr_mod.loc[matched_gt_idx, 'gt_match'] != -1:
                    pass  # The previous best match will be handled below
                groundtr_mod.loc[matched_gt_idx, 'iou'] = iou
                groundtr_mod.loc[matched_gt_idx, 'gt_match'] = p_idx
                groundtr_mod.loc[matched_gt_idx, 'cat'] = 2  # Best match
                prediction_mod.loc[p_idx, 'cat'] = 2  # True positive (best match)
                matched_gt.add(matched_gt_idx)
            else:
                # This is a secondary match (potential oversegmentation)
                prediction_mod.loc[p_idx, 'cat'] = 3  # Mark as oversegmentation
        else:
            # No match found - false positive
            prediction_mod.loc[p_idx, 'cat'] = 1
            false_positives += 1

    # Count oversegmented ground truth polygons (those with multiple matches)
    for idx, gt_poly in enumerate(ground_truth_polys):
        num_matches = groundtr_mod.loc[idx, 'num_match']
        if num_matches > 1:
            oversegmented += 1
            groundtr_mod.loc[idx, 'cat'] = 3  # Mark as oversegmented

        if idx not in matched_gt:
            false_negatives += 1

    # Calculate true positives (unique matches)
    true_positives = len(matched_gt)

    print("True Positives:", true_positives)
    print("False Positives:", false_positives)
    print("False Negatives:", false_negatives)
    print("Oversegmented Ground Truth Polygons:", oversegmented)

    if iou_list:
        avg_iou = sum(iou_list)/len(iou_list)
        print(f"Average IoU: {avg_iou:.4f}")
    else:
        print("Average IoU: 0.0 (no matches)")

    prediction_mod.to_file(prcat_path)
    groundtr_mod.to_file(gtcat_path)

    return true_positives, false_positives, false_negatives, oversegmented

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L0.gpkg"
gt_in = "ls_plot_0_gt.gpkg"

# out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
# out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"
out_pr = site_path + "out_pr/" + "under_pr" + "_out.shp"
out_gt = site_path + "out_gt/" + "under_gt" + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
# Load ground truth shapefile
pr_crown = gpd.read_file(pr_file)
gt_crown = gpd.read_file(gt_file)

In [ ]:
cf_matrix = prediction_evaluation4(pr_crown, gt_crown, out_pr, out_gt)

True Positives: 29
False Positives: 24
False Negatives: 17
Oversegmented Ground Truth Polygons: 5
Average IoU: 0.8183


In [ ]:
out_file = gpd.read_file(out_gt)

In [ ]:
def prediction_evaluation_ov(prediction, ground_truth, prcat_path, gtcat_path, iou_threshold=0.5, ov_iou_th=0.1):
    """
    Compare prediction results and ground truth polygons and returns confusion matrix values.
    It also creates a new column for the category of the polygon for visualization purpose.
    Adds oversegmentation detection.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        prcat_path (String): categorized prediction shapefile output filename and path
        gtcat_path (String): categorized ground truth shapefile output filename and path
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).

    Returns:
        tuple: confusion matrix values (TP, FP, FN), and oversegmentation count.
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["iou"] = 0.0 # iou column
    prediction_mod["gt_match"]= -1 # match column
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["iou"] = 0.0 # iou column
    groundtr_mod["gt_match"]= -1 # match column
    groundtr_mod["num_match"] = 0 # count of matching predictions

    # Track matches
    matched_gt = set()
    duplicated_polys = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    oversegmented = 0

    iou_list = []

    for p_idx, pred_poly in enumerate(predicted_polys):
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0


            if iou >= ov_iou_th:
                groundtr_mod.loc[idx, 'num_match'] += 1
            # Check if IoU exceeds threshold
            if iou >= iou_threshold:
                prediction_mod.loc[p_idx, 'iou'] = max(prediction_mod.loc[p_idx, 'iou'], iou)
                prediction_mod.loc[p_idx, 'gt_match'] = idx
                # groundtr_mod.loc[idx, 'num_match'] += 1

    # Second pass: categorize predictions and count oversegmentation
    for p_idx, pred_poly in enumerate(predicted_polys):
        matched_gt_idx = prediction_mod.loc[p_idx, 'gt_match']
        if matched_gt_idx != -1:  # If this prediction matches a ground truth
            iou = prediction_mod.loc[p_idx, 'iou']
            iou_list.append(iou)

            # Check if this is the best match for the ground truth
            if groundtr_mod.loc[matched_gt_idx, 'iou'] < iou:
                # If there was a previous best match, it might now be an oversegmentation
                if groundtr_mod.loc[matched_gt_idx, 'gt_match'] != -1:
                    pass  # The previous best match will be handled below
                groundtr_mod.loc[matched_gt_idx, 'iou'] = iou
                groundtr_mod.loc[matched_gt_idx, 'gt_match'] = p_idx
                groundtr_mod.loc[matched_gt_idx, 'cat'] = 2  # Best match
                prediction_mod.loc[p_idx, 'cat'] = 2  # True positive (best match)
                matched_gt.add(matched_gt_idx)
            else:
                # This is a secondary match (potential oversegmentation)
                prediction_mod.loc[p_idx, 'cat'] = 3  # Mark as oversegmentation
        else:
            # No match found - false positive
            prediction_mod.loc[p_idx, 'cat'] = 1
            false_positives += 1

    # Count oversegmented ground truth polygons (those with multiple matches)
    for idx, gt_poly in enumerate(ground_truth_polys):
        num_matches = groundtr_mod.loc[idx, 'num_match']
        if num_matches > 1:
            oversegmented += 1
            groundtr_mod.loc[idx, 'cat'] = 3  # Mark as oversegmented

        if idx not in matched_gt:
            false_negatives += 1

    # Calculate true positives (unique matches)
    true_positives = len(matched_gt)

    print("True Positives:", true_positives)
    print("False Positives:", false_positives)
    print("False Negatives:", false_negatives)
    print("Oversegmented Ground Truth Polygons:", oversegmented)

    if iou_list:
        avg_iou = sum(iou_list)/len(iou_list)
        print(f"Average IoU: {avg_iou:.4f}")
    else:
        print("Average IoU: 0.0 (no matches)")

    prediction_mod.to_file(prcat_path)
    groundtr_mod.to_file(gtcat_path)

    return true_positives, false_positives, false_negatives, oversegmented

In [ ]:
cf_matrix = prediction_evaluation_ov(pr_crown, gt_crown, out_pr, out_gt)

True Positives: 29
False Positives: 24
False Negatives: 17
Oversegmented Ground Truth Polygons: 5
Average IoU: 0.8183


In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ov_ud/"
gt_path = site_path + "ov_ud/"

pr_in = "ov_pr.gpkg"
gt_in = "ov_gt.gpkg"

# out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
# out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"
out_pr = site_path + "out_pr/" + "under_pr" + "_out.shp"
out_gt = site_path + "out_gt/" + "under_gt" + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in

In [ ]:
gt_crown = gpd.read_file(gt_file)
pr_crown = gpd.read_file(pr_file)

In [ ]:
cf_matrix = prediction_evaluation_ov(pr_crown, gt_crown, out_pr, out_gt)

True Positives: 0
False Positives: 2
False Negatives: 5
Oversegmented Ground Truth Polygons: 0
Average IoU: 0.0 (no matches)


In [ ]:
out_file = gpd.read_file(out_pr)
out_file.head(40)

,id,cat,iou,gt_match,geometry
0,NaN,1,0.0,-1,"POLYGON ((220585.157 82277.277, 220584.861 822..."
1,NaN,1,0.0,-1,"POLYGON ((220604.479 82215.174, 220604.184 822..."


### Over and Under

In [ ]:
def prediction_evaluation_ov_ud_fix(prediction, ground_truth, prcat_path, gtcat_path, iou_threshold=0.5, ov_iou_th=0.1):
    """
    Compare prediction results and ground truth polygons and returns confusion matrix values.
    It also creates a new column for the category of the polygon for visualization purpose.
    Adds oversegmentation detection.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        prcat_path (String): categorized prediction shapefile output filename and path
        gtcat_path (String): categorized ground truth shapefile output filename and path
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).

    Returns:
        tuple: confusion matrix values (TP, FP, FN), and oversegmentation count.
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["iou"] = 0.0 # iou column
    prediction_mod["gt_match"]= -1 # match column
    prediction_mod["num_match"] = 0  # How many GT this prediction matches
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["iou"] = 0.0 # iou column
    groundtr_mod["gt_match"]= -1 # match column
    groundtr_mod["num_match"] = 0 # count of matching predictions

    # Track matches
    matched_gt = set()
    duplicated_polys = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    oversegmented = 0
    undersegmented = 0

    iou_list = []

    for p_idx, pred_poly in enumerate(predicted_polys):
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0


            if iou >= ov_iou_th:
                groundtr_mod.loc[idx, 'num_match'] += 1
                prediction_mod.at[p_idx, "num_match"] += 1
            # Check if IoU exceeds threshold
            if iou >= iou_threshold:
                prediction_mod.loc[p_idx, 'iou'] = max(prediction_mod.loc[p_idx, 'iou'], iou)
                prediction_mod.loc[p_idx, 'gt_match'] = idx
                # groundtr_mod.loc[idx, 'num_match'] += 1

    # Second pass: categorize predictions and count oversegmentation
    for p_idx, pred_poly in enumerate(predicted_polys):
        matched_gt_idx = prediction_mod.loc[p_idx, 'gt_match']
        if matched_gt_idx != -1:  # If this prediction matches a ground truth
            iou = prediction_mod.loc[p_idx, 'iou']
            iou_list.append(iou)

            # Check if this is the best match for the ground truth
            if groundtr_mod.loc[matched_gt_idx, 'iou'] < iou:
                # If there was a previous best match, it might now be an oversegmentation
                if groundtr_mod.loc[matched_gt_idx, 'gt_match'] != -1:
                    pass  # The previous best match will be handled below
                groundtr_mod.loc[matched_gt_idx, 'iou'] = iou
                groundtr_mod.loc[matched_gt_idx, 'gt_match'] = p_idx
                groundtr_mod.loc[matched_gt_idx, 'cat'] = 2  # Best match
                prediction_mod.loc[p_idx, 'cat'] = 2  # True positive (best match)
                matched_gt.add(matched_gt_idx)
            else:
                # This is a secondary match (potential oversegmentation)
                prediction_mod.loc[p_idx, 'cat'] = 3  # Mark as oversegmentation
        else:
            # No match found - false positive
            prediction_mod.loc[p_idx, 'cat'] = 1
            false_positives += 1

    # Count oversegmented ground truth polygons (those with multiple matches)
    for idx, gt_poly in enumerate(ground_truth_polys):
        num_matches = groundtr_mod.loc[idx, 'num_match']
        if num_matches > 1:
            oversegmented += 1
            groundtr_mod.loc[idx, 'cat'] = 3  # oversegmented

        if idx not in matched_gt:
            false_negatives += 1

    for idx, pr_poly in enumerate(predicted_polys):
        num_matches = prediction_mod.loc[idx, 'num_match']
        if num_matches > 1:
            undersegmented += 1
            prediction_mod.loc[idx, 'cat'] = 4  # undersegmented


    # Calculate true positives (unique matches)
    true_positives = len(matched_gt)

    print("True Positives:", true_positives)
    print("False Positives:", false_positives)
    print("False Negatives:", false_negatives)
    print("Oversegmented:", oversegmented)
    print("Undersegmented:", undersegmented)

    if iou_list:
        avg_iou = sum(iou_list)/len(iou_list)
        print(f"Average IoU: {avg_iou:.4f}")
    else:
        print("Average IoU: 0.0 (no matches)")

    prediction_mod.to_file(prcat_path)
    groundtr_mod.to_file(gtcat_path)

    return true_positives, false_positives, false_negatives, oversegmented

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ov_ud/"
gt_path = site_path + "ov_ud/"

pr_in = "ud_pr2.gpkg"
gt_in = "ud_gt.gpkg"

out_pr = site_path + "out_pr/" + "udpr" + "_out.shp"
out_gt = site_path + "out_gt/" + "udgt" + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in
gt_crown = gpd.read_file(gt_file)
pr_crown = gpd.read_file(pr_file)

In [ ]:
cf_matrix = prediction_evaluation_ov_ud_fix(pr_crown, gt_crown, out_pr, out_gt)

True Positives: 0
False Positives: 1
False Negatives: 3
Oversegmented: 0
Undersegmented: 1
Average IoU: 0.0 (no matches)


In [ ]:
gpd.read_file(out_pr).head()

,id,cat,iou,gt_match,num_matche,geometry
0,NaN,2,1.0,0,1,"POLYGON ((220595.215 82237.198, 220594.886 822..."
1,NaN,2,1.0,1,1,"POLYGON ((220604.738 82237.845, 220604.41 8223..."
2,NaN,2,1.0,2,1,"POLYGON ((220588.318 82235.245, 220588.318 822..."


### Over and Under func

In [ ]:
def over_under(prediction, ground_truth, iou_threshold=0.5, ovud_iou_th=0.1):
    """
    Check occurence of oversegmentation and undersegmentation of the ground truth polygons.
    Creates a new column for the category of the polygon for visualization purpose.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).
        ovud_iou_th (float): IoU threshold to consider a oversegmentation or undersegmentation (default = 0.1).

    Returns:
        tuple: confusion matrix values (TP, FP, FN), and oversegmentation count.
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["num_match"] = 0  # How many GT this prediction matches
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["num_match"] = 0 # count of matching predictions

    oversegmented = 0
    undersegmented = 0

    iou_list = []

    for p_idx, pred_poly in enumerate(predicted_polys):
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0


            if iou >= ovud_iou_th:
                groundtr_mod.loc[idx, 'num_match'] += 1
                prediction_mod.at[p_idx, "num_match"] += 1

    for idx, gt_poly in enumerate(ground_truth_polys):
        num_matches = groundtr_mod.loc[idx, 'num_match']
        if num_matches > 1:
            oversegmented += 1
            groundtr_mod.loc[idx, 'cat'] = 3  # oversegmented

    for idx, pr_poly in enumerate(predicted_polys):
        num_matches = prediction_mod.loc[idx, 'num_match']
        if num_matches > 1:
            undersegmented += 1
            prediction_mod.loc[idx, 'cat'] = 4  # undersegmented

    print("Oversegmented:", oversegmented)
    print("Undersegmented:", undersegmented)

    return oversegmented, undersegmented

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ov_ud/"
gt_path = site_path + "ov_ud/"

pr_in = "ud_pr.gpkg"
gt_in = "ud_gt.gpkg"

out_pr = site_path + "out_pr/" + "udpr" + "_out.shp"
out_gt = site_path + "out_gt/" + "udgt" + "_out.shp"

In [ ]:
site_path = "/content/drive/MyDrive/Research/results/"
pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

pr_in = "ls_resnet_L0.gpkg"
gt_in = "ls_plot_0_gt.gpkg"

# out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
# out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"
out_pr = site_path + "out_pr/" + "under_pr" + "_out.shp"
out_gt = site_path + "out_gt/" + "under_gt" + "_out.shp"

In [ ]:
pr_file = pr_path + pr_in
gt_file = gt_path + gt_in
gt_crown = gpd.read_file(gt_file)
pr_crown = gpd.read_file(pr_file)

In [ ]:
cf_matrix = over_under(pr_crown, gt_crown)

Oversegmented: 5
Undersegmented: 2


# Fix evaluations

In [165]:
import geopandas as gpd

def calculate_precision_recall_f1(confussion_matix):
    """
    Calculate precisoin, recall, and F1 based on the confussion matrix (TP, FP, FN)

    Args:
        confussion_matrix (tuple): confussion matrix values consisting TP, FP, FN.

    Returns:
        tuple: Precision and Recall scores.
    """
    # Confussion matrix values slicing
    true_positives, false_positives, false_negatives = confussion_matix

    # Calculate precision, recall, and F1
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

In [166]:
from shapely.ops import unary_union

def calculate_iou(prediction, ground_truth):
    """
    Calculate IoU between prediction and ground truth polygons.

    Args:
        prediction (GeoDataFrame): Predicted polygons.
        ground_truth (GeoDataFrame): Ground truth polygons.

    Returns:
        float: IoU score between prediction and ground truth.
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    # Merge all polygons in prediction and ground truth
    merged_prediction = unary_union(prediction.geometry)
    merged_ground_truth = unary_union(ground_truth.geometry)

    # Calculate intersection and union areas
    intersection_area = merged_prediction.intersection(merged_ground_truth).area
    union_area = merged_prediction.union(merged_ground_truth).area

    # IoU calculation
    iou = intersection_area / union_area if union_area > 0 else 0

    return iou

In [167]:
def prediction_evaluation2(prediction, ground_truth, prcat_path, gtcat_path, iou_threshold=0.5):
    """
    Compare prediction results and ground truth polygons and returns ocnfussion matrix values.
    It aslo creates a new column for the category of the polygon for the visualization purpose.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        prcat_path (String): categorized prediction shapefile output filename and path
        gtcat_path (String): categorized grount truth shapefile output filename and path
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).

    Returns:
        tuple: confussion matrix values (TP, FP, FN).
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    # Convert polygons to lists for pairwise comparison
    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["iou"] = 0.0 # iou column
    prediction_mod["gt_match"]= -1 # match column
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["iou"] = 0.0 # iou column
    groundtr_mod["gt_match"]= -1 # match column


    # Track matches
    matched_gt = set()  # Track matched ground truth indices
    duplicated_polys = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    iou_list = []

    # Check each predicted polygon
    for p_idx, pred_poly in enumerate(predicted_polys):
        matched = False
        duplicated = False
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0
            prediction_mod.loc[p_idx, 'iou'] = iou # For IoU

            # Check if IoU exceeds threshold
            if iou >= iou_threshold:
                matched = True
                iou_list.append(iou)
                matched_gt.add(idx)  # Mark this ground truth as matched
                groundtr_mod.loc[idx, 'cat'] = 2 # Category 2 for Matched
                if groundtr_mod.loc[idx, 'gt_match'] != -1:
                    duplicated = True
                if groundtr_mod.loc[idx, 'gt_match'] == -1 or groundtr_mod.loc[idx, 'iou'] < iou: #Store the matching prediction
                    groundtr_mod.loc[idx, 'iou'] = iou
                    groundtr_mod.loc[idx, 'gt_match'] = p_idx #
                    # duplicated_polys += 1
                prediction_mod.loc[p_idx, 'gt_match'] = idx # Store the matching ground truth polygon
                break


        if matched:
            true_positives += 1
            prediction_mod.loc[p_idx, 'cat'] = 2 # Category 2 for TP
        else:
            false_positives += 1
            # prediction_mod.loc[p_idx, 'cat'] = 1 # Category 1 for FP

        if duplicated:
            duplicated_polys += 1

    # Re-Count the True positives and False positives considering the duplication
    true_positives = true_positives - duplicated_polys
    false_positives = false_positives

    # Count false negatives (unmatched ground truth polygons)
    false_negatives = len(ground_truth_polys) - len(matched_gt)

    print("True Positives", true_positives)
    print("False Positives", false_positives)
    print("False Negatives", false_negatives)
    # print(iou_list)
    avg_iou = sum(iou_list)/len(iou_list)
    print(f"Average IoU: {avg_iou:.4f}")

    prediction_mod.to_file(prcat_path)
    groundtr_mod.to_file(gtcat_path)

    return true_positives, false_positives, false_negatives

In [168]:
def over_under(prediction, ground_truth, iou_threshold=0.5, ovud_iou_th=0.1):
    """
    Check occurence of oversegmentation and undersegmentation of the ground truth polygons.
    Creates a new column for the category of the polygon for visualization purpose.

    Args:
        prediction (GeoDataFrame): predicted polygons.
        ground_truth (GeoDataFrame): ground truth polygons.
        iou_threshold (float): IoU threshold to consider a match (default = 0.5).
        ovud_iou_th (float): IoU threshold to consider a oversegmentation or undersegmentation (default = 0.1).

    Returns:
        tuple: confusion matrix values (TP, FP, FN), and oversegmentation count.
    """
    # Convert prediction CRS to ground truth CRS
    if prediction.crs != ground_truth.crs:
        prediction = prediction.to_crs(ground_truth.crs)

    predicted_polys = list(prediction.geometry)
    ground_truth_polys = list(ground_truth.geometry)

    prediction_mod = prediction.copy()
    prediction_mod["cat"] = 0 # category column
    prediction_mod["num_match"] = 0  # How many GT this prediction matches
    groundtr_mod = ground_truth.copy()
    groundtr_mod["cat"] = 0 # category column
    groundtr_mod["num_match"] = 0 # count of matching predictions

    oversegmented = 0
    undersegmented = 0

    iou_list = []

    for p_idx, pred_poly in enumerate(predicted_polys):
        for idx, gt_poly in enumerate(ground_truth_polys):
            # Calculate IoU
            intersection_area = pred_poly.intersection(gt_poly).area
            union_area = pred_poly.union(gt_poly).area
            iou = intersection_area / union_area if union_area > 0 else 0


            if iou >= ovud_iou_th:
                groundtr_mod.loc[idx, 'num_match'] += 1
                prediction_mod.at[p_idx, "num_match"] += 1

    for idx, gt_poly in enumerate(ground_truth_polys):
        num_matches = groundtr_mod.loc[idx, 'num_match']
        if num_matches > 1:
            oversegmented += 1
            groundtr_mod.loc[idx, 'cat'] = 3  # oversegmented

    for idx, pr_poly in enumerate(predicted_polys):
        num_matches = prediction_mod.loc[idx, 'num_match']
        if num_matches > 1:
            undersegmented += 1
            prediction_mod.loc[idx, 'cat'] = 4  # undersegmented

    print("Oversegmented:", oversegmented)
    print("Undersegmented:", undersegmented)

    return oversegmented, undersegmented

### Over Under

In [161]:
num_code = ["0", "2", "4"]

In [162]:
ls_name = [
    "ls_resnet_L",
    "ls_resnext_L",
    "ls_resnet_pr_L",
    "ls_resnext_pr_L"]

In [163]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "ls_res/"
gt_path = site_path + "ground_truth/"

In [164]:
import warnings
warnings.filterwarnings('ignore')

In [169]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "ls_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)
    oversegmented, undersegmented = over_under(pr_crown, gt_crown)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"Oversegmented: {oversegmented}")
    print(f"Undersegmented: {undersegmented}")


    print("================================")


ls_resnet_L
Results for 0
True Positives 29
False Positives 24
False Negatives 17
Average IoU: 0.8183
Oversegmented: 5
Undersegmented: 2
IoU: 0.6651
Precision: 0.5472
Recall: 0.6304
F1 Score: 0.5859
Oversegmented: 5
Undersegmented: 2
Results for 2
True Positives 39
False Positives 21
False Negatives 37
Average IoU: 0.8143
Oversegmented: 9
Undersegmented: 5
IoU: 0.5001
Precision: 0.6500
Recall: 0.5132
F1 Score: 0.5735
Oversegmented: 9
Undersegmented: 5
Results for 4
True Positives 11
False Positives 21
False Negatives 11
Average IoU: 0.7951
Oversegmented: 6
Undersegmented: 4
IoU: 0.3267
Precision: 0.3438
Recall: 0.5000
F1 Score: 0.4074
Oversegmented: 6
Undersegmented: 4

ls_resnext_L
Results for 0
True Positives 30
False Positives 16
False Negatives 16
Average IoU: 0.7924
Oversegmented: 6
Undersegmented: 4
IoU: 0.6605
Precision: 0.6522
Recall: 0.6522
F1 Score: 0.6522
Oversegmented: 6
Undersegmented: 4
Results for 2
True Positives 41
False Positives 20
False Negatives 35
Average IoU: 0.

### Otomate JW

In [170]:
num_code = ["2"]

In [171]:
ls_name = [
    "jw_resnet_P",
    "jw_resnext_P",
    "jw_resnet_pr_P",
    "jw_resnext_pr_P"]

In [172]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "jw_res/"
gt_path = site_path + "ground_truth/"

In [174]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "jw_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)
    oversegmented, undersegmented = over_under(pr_crown, gt_crown)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"Oversegmented: {oversegmented}")
    print(f"Undersegmented: {undersegmented}")

    print("================================")


jw_resnet_P
Results for 2
True Positives 35
False Positives 12
False Negatives 31
Average IoU: 0.8045
Oversegmented: 8
Undersegmented: 5
IoU: 0.4733
Precision: 0.7447
Recall: 0.5303
F1 Score: 0.6195
Oversegmented: 8
Undersegmented: 5

jw_resnext_P
Results for 2
True Positives 44
False Positives 20
False Negatives 22
Average IoU: 0.8005
Oversegmented: 7
Undersegmented: 3
IoU: 0.5971
Precision: 0.6875
Recall: 0.6667
F1 Score: 0.6769
Oversegmented: 7
Undersegmented: 3

jw_resnet_pr_P
Results for 2
True Positives 32
False Positives 4
False Negatives 34
Average IoU: 0.8293
Oversegmented: 1
Undersegmented: 1
IoU: 0.4101
Precision: 0.8889
Recall: 0.4848
F1 Score: 0.6275
Oversegmented: 1
Undersegmented: 1

jw_resnext_pr_P
Results for 2
True Positives 30
False Positives 4
False Negatives 36
Average IoU: 0.8280
Oversegmented: 1
Undersegmented: 4
IoU: 0.4155
Precision: 0.8824
Recall: 0.4545
F1 Score: 0.6000
Oversegmented: 1
Undersegmented: 4


### Otomate LS - Exp ResNet Pr

In [175]:
num_code = ["0", "2", "4"]

In [176]:
ls_name = [
    "ls_resnet_pr7_L",
    "ls_resnet_pr6_L"]

In [177]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "ls_exp/"
gt_path = site_path + "ground_truth/"

In [178]:
import warnings
warnings.filterwarnings('ignore')

In [179]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "ls_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)
    oversegmented, undersegmented = over_under(pr_crown, gt_crown)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"Oversegmented: {oversegmented}")
    print(f"Undersegmented: {undersegmented}")

    print("================================")


ls_resnet_pr7_L
Results for 0
True Positives 20
False Positives 0
False Negatives 26
Average IoU: 0.7956
Oversegmented: 1
Undersegmented: 4
IoU: 0.5954
Precision: 1.0000
Recall: 0.4348
F1 Score: 0.6061
Oversegmented: 1
Undersegmented: 4
Results for 2
True Positives 21
False Positives 3
False Negatives 55
Average IoU: 0.8467
Oversegmented: 1
Undersegmented: 4
IoU: 0.4354
Precision: 0.8750
Recall: 0.2763
F1 Score: 0.4200
Oversegmented: 1
Undersegmented: 4
Results for 4
True Positives 9
False Positives 7
False Negatives 13
Average IoU: 0.7766
Oversegmented: 2
Undersegmented: 2
IoU: 0.3918
Precision: 0.5625
Recall: 0.4091
F1 Score: 0.4737
Oversegmented: 2
Undersegmented: 2

ls_resnet_pr6_L
Results for 0
True Positives 16
False Positives 2
False Negatives 30
Average IoU: 0.7960
Oversegmented: 0
Undersegmented: 3
IoU: 0.5296
Precision: 0.8889
Recall: 0.3478
F1 Score: 0.5000
Oversegmented: 0
Undersegmented: 3
Results for 2
True Positives 24
False Positives 3
False Negatives 52
Average IoU: 0

### Otomate LS - Exp ResNeXt Pr

In [180]:
num_code = ["0", "2", "4"]

In [181]:
ls_name = [
    "ls_resnext_pr17_L",
    "ls_resnext_pr15_L",
    "ls_resnext_pr12_L"]

In [182]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "ls_exp/"
gt_path = site_path + "ground_truth/"

In [183]:
import warnings
warnings.filterwarnings('ignore')

In [184]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "ls_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)
    oversegmented, undersegmented = over_under(pr_crown, gt_crown)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"Oversegmented: {oversegmented}")
    print(f"Undersegmented: {undersegmented}")

    print("================================")


ls_resnext_pr17_L
Results for 0
True Positives 19
False Positives 1
False Negatives 27
Average IoU: 0.8286
Oversegmented: 0
Undersegmented: 2
IoU: 0.5932
Precision: 0.9500
Recall: 0.4130
F1 Score: 0.5758
Oversegmented: 0
Undersegmented: 2
Results for 2
True Positives 20
False Positives 4
False Negatives 56
Average IoU: 0.8316
Oversegmented: 0
Undersegmented: 2
IoU: 0.3942
Precision: 0.8333
Recall: 0.2632
F1 Score: 0.4000
Oversegmented: 0
Undersegmented: 2
Results for 4
True Positives 8
False Positives 2
False Negatives 14
Average IoU: 0.8326
Oversegmented: 0
Undersegmented: 0
IoU: 0.3280
Precision: 0.8000
Recall: 0.3636
F1 Score: 0.5000
Oversegmented: 0
Undersegmented: 0

ls_resnext_pr15_L
Results for 0
True Positives 21
False Positives 3
False Negatives 25
Average IoU: 0.8273
Oversegmented: 1
Undersegmented: 1
IoU: 0.4775
Precision: 0.8750
Recall: 0.4565
F1 Score: 0.6000
Oversegmented: 1
Undersegmented: 1
Results for 2
True Positives 27
False Positives 8
False Negatives 49
Average Io

### Otomate JW - Exp ResNeXt Pr

In [185]:
num_code = ["2"]

In [186]:
ls_name = [
    "jw_resnext_pr17_P",
    "jw_resnext_pr15_P",
    "jw_resnext_pr12_P"]

In [187]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "jw_exp/"
gt_path = site_path + "ground_truth/"

In [188]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "jw_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)
    oversegmented, undersegmented = over_under(pr_crown, gt_crown)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"Oversegmented: {oversegmented}")
    print(f"Undersegmented: {undersegmented}")

    print("================================")


jw_resnext_pr17_P
Results for 2
True Positives 30
False Positives 2
False Negatives 36
Average IoU: 0.8478
Oversegmented: 0
Undersegmented: 2
IoU: 0.4641
Precision: 0.9375
Recall: 0.4545
F1 Score: 0.6122
Oversegmented: 0
Undersegmented: 2

jw_resnext_pr15_P
Results for 2
True Positives 30
False Positives 4
False Negatives 36
Average IoU: 0.8347
Oversegmented: 1
Undersegmented: 5
IoU: 0.4436
Precision: 0.8824
Recall: 0.4545
F1 Score: 0.6000
Oversegmented: 1
Undersegmented: 5

jw_resnext_pr12_P
Results for 2
True Positives 36
False Positives 8
False Negatives 30
Average IoU: 0.8091
Oversegmented: 5
Undersegmented: 2
IoU: 0.4822
Precision: 0.8182
Recall: 0.5455
F1 Score: 0.6545
Oversegmented: 5
Undersegmented: 2


### Otomate JW - Exp ResNet Pr

In [189]:
num_code = ["2"]

In [190]:
ls_name = [
    "jw_resnet_pr7_P",
    "jw_resnet_pr6_P"]

In [191]:
site_path = "/content/drive/MyDrive/Research/results/"

pr_path = site_path + "jw_exp/"
gt_path = site_path + "ground_truth/"

In [192]:
for ls in ls_name:
  print("")
  print(ls)
  for num in num_code:
    print("Results for " + num)

    pr_in = ls + num + ".gpkg"
    gt_in = "jw_plot_" + num + "_gt" + ".gpkg"

    pr_file = pr_path + pr_in
    gt_file = gt_path + gt_in

    out_pr = site_path + "out_pr/" + pr_in[:pr_in.index(".")] + "_out.shp"
    out_gt = site_path + "out_gt/" + gt_in[:gt_in.index(".")] + "_out.shp"

    pr_crown = gpd.read_file(pr_file)
    gt_crown = gpd.read_file(gt_file)

    iou = calculate_iou(pr_crown, gt_crown)
    cf_matrix = prediction_evaluation2(pr_crown, gt_crown, out_pr, out_gt)
    precision, recall, f1_score = calculate_precision_recall_f1(cf_matrix)
    oversegmented, undersegmented = over_under(pr_crown, gt_crown)

    print(f"IoU: {iou:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")
    print(f"Oversegmented: {oversegmented}")
    print(f"Undersegmented: {undersegmented}")

    print("================================")


jw_resnet_pr7_P
Results for 2
True Positives 28
False Positives 2
False Negatives 38
Average IoU: 0.8218
Oversegmented: 2
Undersegmented: 3
IoU: 0.4805
Precision: 0.9333
Recall: 0.4242
F1 Score: 0.5833
Oversegmented: 2
Undersegmented: 3

jw_resnet_pr6_P
Results for 2
True Positives 25
False Positives 4
False Negatives 41
Average IoU: 0.8343
Oversegmented: 3
Undersegmented: 4
IoU: 0.3969
Precision: 0.8621
Recall: 0.3788
F1 Score: 0.5263
Oversegmented: 3
Undersegmented: 4
